In [6]:
day = 15
from IPython.display import IFrame
url = "https://adventofcode.com/2022/day/%d"%(day)
IFrame(src=url, width=1000, height=600)

In [8]:
import urllib.request
import sys
from timeit import default_timer as timer
import itertools


def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2022):
    with open('sessionID') as f:
        sessionId = f.readlines()[0]
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data

def read_file(filename):
    # read file
    with open(filename) as f:
        lines = f.readlines()
    output = [str.rstrip(line) for line in lines]    
    return output


In [10]:
test = False
data = get_data(day)
if test:
    data = read_file('test.txt')
    
import re
from scipy.spatial.distance import cityblock

data = [re.findall('-?\d+', val) for val in data] 
s = []
b = []
for line in data:
    s.append((int(line[0]),int(line[1])))
    b.append((int(line[2]),int(line[3])))

SB = []
for ss,bb in zip(s,b):
        man_dist = cityblock(ss,bb)
        SB.append((ss,bb,man_dist))

In [15]:

def fill_row_part1(row_of_interest, SB = SB):
    empty = []
    for ss,bb,dd in SB:
        (sx,sy) = ss

        # just add values that will effect row of interest 
        dist_to_interest = dd - abs(sy - row_of_interest)
        if dist_to_interest < 0:
            continue

        # collect ranges for now
        # distance to interest gives us taper
        empty.extend(range(sx-dist_to_interest,sx+dist_to_interest))
        # check if S in row_of_interest, then get rid of that
    return empty

def fill_row_part2(row_of_interest, SB=SB):
    empty = []
    for ss,bb,dd in SB:
 
        (sx,sy) = ss

        # just add values that will effect row of interest 
        dist_to_interest = dd - abs(sy - row_of_interest)
        if dist_to_interest < 0:
            continue

        # collect ranges for now
        # distance to interest gives us taper
        #print(dd,dist_to_interest)
        empty.append((sx-dist_to_interest,sx+dist_to_interest))
        # check if S in row_of_interest, then get rid of that
    empty.sort()
    interval = [[empty[0][0],empty[0][1]]]
    for lo,hi in empty[1:]:
        [ilo, ihi] = interval[-1]

        if lo > ihi + 1:
            interval.append([lo, hi])
            continue
        
        interval[-1][1] = max(ihi, hi)        
        
    return interval

In [16]:
row_of_interest = 2000000 
if test:
    row_of_interest = 10
    
empty = fill_row_part1(row_of_interest)
print(len(set(empty)))

In [17]:
# we are looking for a blanks space in box 
# for each row find highest interval
# if the blank space in on that row, then
# the lower value of that highest interval 
# will be bigger than edge of the box (in this case 0)

max_ord = 4000000
min_ord = 0
done = False
for row_to_search in range(0,max_ord + 1):
    q = fill_row_part2(row_to_search)
    x = 0
    for lo, hi in q:
        
        if x < lo:
            print(x * max_ord + row_to_search)
            done = True
        x = max(x, hi + 1)
        if x > max_ord:
            break
    if done:
        break

10649103160102
